# Analysis Summary

Our goal from this analysis will be to predict the outcome of March Madness games. To do this, we will be using a random forest classifer that takes in historical data from NCAA tourney matchups as input and outputs predictions for the winning team for each game in an input set of March Madness games. 

### Baseline Model
The baseline model we will use to compare our model's results against will be solely tied to a team's RPI. Our baseline model poses the following hypothesis:

    That in any NCAA tournament game, the team with the lower RPI will win the game.
  
Intuitively, this is a reasonable prediction. RPI (Ratings Percentage Index), ranks teams based on their wins, losses, and strength of schedule for the past season. If team A has a lower RPI than team B at the end of a season, it's generally considered that team A's performance throughout the season has been at a higher level than team B's. For this reason, if we knew nothing else about the two teams, predicting the outcome based on RPI is a good starting point. 
### Random Forest Approach
We would like to build on our baseline model and see if we can develop an approach that more accurately predicts the outcome of games. One of the most exciting parts of March Madness is the array of upsets that occur throughout the tournament. In general, upsets occur when a team with a higher RPI beats a team with a lower RPI. We would like to create a model that performs better than our baseline model by more accurately predicting the outcome of games, in particular predicting when upsets occur. For us to accomplish this, we can utilize RPI as well as some additional attributes that provide more information surrounding each team's level of performance in the past season. These factors can be used to build a random forest in order to: 
    1. Predict when an upset is going to occur in March Madness
    2. Identify which factors are correlated to predicting the outcome of a game

We will build a random forest by passing as input a data frame where each row corresponds to an NCAA tournament game. Each row will contain data regarding each team's yearly averages and totals in statistical categories, RPI, the game's outcome, and whether or not the team with the lowest RPI won. This last piece of information will be our dependent variable. The random forest will utilize features in our training data describing the winning team and losing team's performance during the season, in order to learn which factors are tied to predicting the outcome of a game. Once the random forest has been trained, tournament data where the outcome of the each game has been excluded can be used as input to the model to generate a set of predictions for each game in that year's tournament and test the accuracy of our random forest classifer.

### Data Sources Overview
The data the was used to perform this analysis came from the Google Cloud & NCAA® ML Competition 2018-Men's Kaggle competition (Insert Link). The following is a brief summary of all the datasets that were used:
- NCAATourneyCompactResults: Contains records from each NCAA tournament game from 1985-2017, including score and region.  
- Teams: Contains information for each Division 1 (D1) basketball team including an ID, name, first and most recent year playing D1 basketball. 
- MasseyOrdinals_Prelim2018: Contains data from 2003-2017 surrounding each D1 team's rank from various rankings sources throughout. 
- RegularSeasonDetailedResults: Contains similar information as the NCAATourneyCompactResults dataset, with the addition that each row will also contain the totals in a variety of statistical categories for the winning team and losing team. These are categories that are often found in a boxscore. 

A more in depth description of each of the datasets that were used and additional datasets provided by Kaggle can be found at https://www.kaggle.com/c/mens-machine-learning-competition-2018/data


### Data Cleansing & Preparation

In order to create our random forest, we need to complete the following:
    1. Settle on a set of statistics to evaluate the performance of each team. 
    2. Create a data frame that represents a matchup that occurred in an NCAA tournament and contains our set of statistics for each team in the matchup 

To accomplish the first task, the following statistics were calculated for each team:
    - Defensive Rebounds (DRB)
    - Efficient Field Goal Percentage (EFG)
    - Free Throws Attempted (FTA)
    - Free Throw Percentage (FTP)
    - Margin of Loss (MOL)
    - Margin of Victory (MOV)
    - Offensive Rebounds (ORB)
    - Possessions (POSS)
    - Turnovers (TO)
    - Turnovers Forced (TOF)
    - Defensive Efficiency (dEff)
    - Offensive Effiency (oEff)
    - Rankings Percentage Index (RPI)
    - Conference Tourney Wins (confTournWins)
    - Wins vs. a Tournament Team (winsVsTourney)
    - Number of Games Played (numGamesPlayed)


With the exception of the last four, the yearly pre-tournament averages (regular season and conference tourney play) were found. The last three are cumulative totals, and RPI was found by using the latest ranking given to a team before the tournament. 

By parsing through the 4 datasets listed above, we can calculate the desired information for each team in a March Madness tournament from 2003-2017 as seen below:

In [1]:
matchups = getMatchupData()
matchups.head()

NameError: name 'getMatchupData' is not defined

Each row in the output data frame represents a game played in an March Madness tournament. The 16 statistics that were targeted are used features as well as some additional information to help identify which teams played and when the game occurred. A more detailed look at how the Matchups data frame was created can be seen in the getMatchupData function in the Appendix. 

### Random Forest Creation

The matchups data frame can be used as input to a random forest classifier to output predictions for a subset of NCAA tournament games. When creating a random forest classifer, a few parameters should be considered:

- _Maximum Number of Features_: The maximum number of features that our algorithm uses to create a decision tree that will be a part of our random forest. A value of sqrt(n) where n is the number of features in our input dataset was chosen. This is generally considered to be a good starting point for random forest classifiers because it allows the decision trees that are created to have a strong chance of selecting a unique set of features to train a model with. The more unique our set of decision trees are, the more confident we can be in our random forest's performance on test data. 
- _Number of Trees_: The number of trees in a random forest impacts the classifier's effectiveness for similar reason. If not enough decision trees are chosen, some features may not be included in our model and the classifer's effectiveness is hindered. The more trees in a forest, the more likely we cover the full feature space. To have a strong chance of covering the full feature space, 1000 trees were selected to be in our random forest.

For this project, random forests classifiers were created for two use cases:
    1. Predicting the outcome of all NCAA tournament games from one season between 2003-2017
    2. Predicting the outcome of all NCAA tournament championship games from 2003-2017

In order to accomplish both tasks, two methods were created, getPredictionsChips and getPredictions. Both methods behave similarly, except getPredictionChips uses championship games as its test data set to output predictions and getPredictions uses a specified year as its test data set. In each method, a data set of historical NCAA tournament games is cleaned up by removing any qualitative columns or any column specifying a result from our baseline model. Once the training and test data sets are cleaned, a random forest classifier is created using the training data and predictions are made on the test data using the generated classifier. Each method returns a tuple with the following information: 
- _Output Predictions_ (outputPreds): A dataframe describing the prediction for each matchup, the actual result, and the probability for the predicted outcome to occur
- _Baseline Model Accuracy_ (baselineAcc): The accuracy from our baseline model that predicted a game's outcome based on which team had the lower RPI
- _Random Forest Classifier Accuracy_ (modelAcc): The accuracy from our random forest classifier that predicted a game's outcome on a variety of input features

### TODO: Random Forest Classifier Results vs. Baseline Model
- 15 Year comparison with baseline model
- Championship game predictions vs. baseline model

- Two sets of results to look at
1. Results of chip game analysis
2. Results of per season analysis 

For each set of reslts, we compared the classifier's prediction accuracy to the baseline model's. In all years of games analyzed,our model always outperformed the baseline model. This tells us that we can be smarter when picking games and not simply rely on RPI.

In [416]:
indPredicts = []
outCols = ["Predict", "Actual", "A Name", "A ID", "B Name", "B ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []

# Chip games testing
outputPreds, baselineAcc, modelAcc = getPredictionsChips()
print("Baseline Model Accuracy: {}".format(round(baselineAcc, 2)))
print("Our Model's Accuracy: {}".format(round(modelAcc, 2)))

for row in outputPreds:
    indPredicts.append(row.tolist())
predsDF = pd.DataFrame(indPredicts, columns = outCols)
# predsDF
# pd.DataFrame(indPredicts).to_csv("data/output/chipTestResults.csv", index=False, header=True)
# outputPreds


[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
[0 1 0 0 0 0 1 1 1 1 1 0 0 1 1]
-21.0
15
Baseline Model Accuracy: 0.53
Our Model's Accuracy: 0.6


In [5]:
model = getForestDeltas("2003")
model.setFeatureImportances()
importances = model.featureImportances
model.setPredictions()
outputPreds = model.getModelPredictionsAndProbs()
baselineAcc = model.baselineAcc
modelAcc = model.modelAcc

0.671875

In [58]:
indPredicts = []
outCols = ["Predict", "Actual", "L Name", "L ID", "W Name", "W ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []
models = []
for i in range(2003, 2018):
    model = getForestDeltas(str(i))
    model.setFeatureImportances()
    importances = model.featureImportances
    model.setPredictions()
    outputPreds = model.getModelPredictionsAndProbs()
    baselineAcc = model.baselineAcc
    modelAcc = model.modelAcc
    models.append(model)
    baseAccs.append(round(baselineAcc, 3))
    modelAccs.append(round(modelAcc, 3))

    for row in outputPreds:
        indPredicts.append(row.tolist())

# pd.DataFrame(indPredicts).to_csv("data/output/results_deltas_with_seeds_D.csv", index=False, header=True)
accDF = pd.DataFrame({"Season": range(2003,2018), "Baseline": baseAccs, "RF": modelAccs})
predsDF = pd.DataFrame(indPredicts, columns = outCols)

In [42]:
predsDF

,Predict,Actual,L Name,L ID,W Name,W ID,Prob For,Prob Against
0,0,0,F Dickinson,1192_2016,FL Gulf Coast,1195_2016,0.869,0.131
1,0,1,Pittsburgh,1338_2016,Wisconsin,1458_2016,0.555,0.445
2,0,0,Oklahoma,1328_2016,Villanova,1437_2016,0.865,0.135
3,0,0,Oregon St,1333_2016,VA Commonwealth,1433_2016,0.630,0.370
4,0,1,MTSU,1292_2016,Syracuse,1393_2016,0.522,0.478
5,0,1,S Dakota St,1355_2016,Maryland,1268_2016,0.542,0.458
6,0,1,Texas A&M,1401_2016,Oklahoma,1328_2016,0.571,0.429
7,0,1,Gonzaga,1211_2016,Syracuse,1393_2016,0.526,0.474
8,0,0,Oregon,1332_2016,Oklahoma,1328_2016,0.555,0.445
9,0,0,Arizona,1112_2016,Wichita St,1455_2016,0.527,0.473


In [61]:
accDF

,Season,Baseline,RF
0,2003,0.672,0.703
1,2004,0.734,0.719
2,2005,0.688,0.688
3,2006,0.672,0.672
4,2007,0.797,0.766
5,2008,0.766,0.750
6,2009,0.750,0.703
7,2010,0.672,0.672
8,2011,0.687,0.672
9,2012,0.687,0.731


In [60]:
for model in models:
    df = pd.DataFrame(model.confusionMatrix)
    df.rename(columns={0:'Pred Upset', 1:'Pred No Upset'}, 
                 index={0:'Actual Upset',1:'Actual No Upset'}, 
                 inplace=True)
    print(df)

                 Pred Upset  Pred No Upset
Actual Upset              3             18
Actual No Upset           1             42
                 Pred Upset  Pred No Upset
Actual Upset              2             15
Actual No Upset           3             44
                 Pred Upset  Pred No Upset
Actual Upset              3             17
Actual No Upset           3             41
                 Pred Upset  Pred No Upset
Actual Upset              2             19
Actual No Upset           2             41
                 Pred Upset  Pred No Upset
Actual Upset              3             10
Actual No Upset           5             46
                 Pred Upset  Pred No Upset
Actual Upset              3             12
Actual No Upset           4             45
                 Pred Upset  Pred No Upset
Actual Upset              1             15
Actual No Upset           4             44
                 Pred Upset  Pred No Upset
Actual Upset              3             18
Actual No U

In [55]:

def findChampionshipMatches():
    """
    Read in NCAA tourney matchups and return data frame containing additional column denoting (True/False) if that matchup was a championship game. 
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    ## group by season and with resulting groupby obj, find whether each row equals the dayNum max for each group
    ## store result as column in matchups defining whether championship played that day
    ## able to pass in functions to transform to perform calculations for each group
    matchups["chipGame"] = matchups.groupby(['season'])['dayNum'].transform(max) == matchups['dayNum']
    return matchups

def getPredictionsChips():
    """
    Outputs predictions for all championship games from 2003-2017 using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = findChampionshipMatches()
    cols = list(matchups.columns)
    
    # Create training/test data sets
    train = matchups[matchups["chipGame"] == False]
    test = matchups[matchups["chipGame"] == True]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    ## Draw sample classification tree
    # drawTree(rf, "sampleTree")

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

### Utilize historical matchup data to build RF model. 
def getPredictions(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    cols = list(matchups.columns)
    train = matchups[~matchups["b_id"].str.contains(year)]
    test = matchups[matchups["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    
    train = train.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)
    test = test.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

### Utilize historical matchup data to build RF model. 
def getForestDeltas(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    model = mod.Model()
    matchups = getMatchupData()
    seededMatches = mergeRankings(matchups)
    sortedMatches = sortMatchups(seededMatches)
    deltas = pd.DataFrame(findAllDeltas(sortedMatches))
    
    cols = list(deltas.columns)
    trainOrig = deltas[~deltas["b_id"].str.contains(year)]
    testOrig = deltas[deltas["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(testOrig["baseline"]) / testOrig.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(trainOrig["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(testOrig["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((testOrig["aname"], testOrig["a_id"], testOrig["bname"], testOrig["b_id"]))
    model.trainBaseline = trainLabels
    model.testBaseline = testLabels
    model.testNames = testNames
    
    # Drop qualitative & output columns
    train = trainOrig.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    test = testOrig.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    # Drop insignificant quantitative columns
    train = train.drop(["MOL", "MOV", "POSS", "TO", "TOF", "numGamesPlayed", "RPI"], axis = 1)
    test = test.drop(["MOL", "MOV", "POSS", "TO", "TOF", "numGamesPlayed", "RPI"], axis = 1)
    model.train = train
    model.test = test

    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    model.forest = rf
    return model

def drawTree(rf, treeName):
    dot_data = StringIO()
    export_graphviz(rf.estimators_[0], out_file=dot_data, filled=True, rounded=True, special_characters=True, feature_names=feature_names)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_pdf("{}.pdf".format(treeName))
    
    
def sortRowByRPI(matchup):
    if matchup["lRPI"] > matchup["wRPI"]:
        numCols = matchup.shape[0]
        newOrder = [0, 1, 4, 5, 2, 3] + list(range(22, numCols - 3)) + list(range(6,22)) + [numCols - 2, numCols - 3, numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    return list(matchup.values)

def sortRowBySeed(matchup):
    if matchup["w_seed"] > matchup["l_seed"]:
        numCols = matchup.shape[0]
        newOrder = list(range(6)) + list(range(22, numCols - 3)) + list(range(6,22)) + [numCols - 2, numCols - 3, numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    return list(matchup.values)

def executeSeedTiebreaker(matchup):
    if matchup["w_seed"] == matchup["l_seed"]:
        matchup["baseline"] = matchup["wRPI"] < matchup["lRPI"]
    return matchup

def findDeltasForMatch(matchup):
    numCols = matchup.shape[0]
    return list(matchup.iloc[0:6].values) + list(matchup.iloc[6:22].values - matchup.iloc[22:numCols - 3].values) + [matchup.iloc[numCols - 3] - matchup.iloc[numCols - 2]] + [matchup.iloc[numCols - 1]]
        

In [56]:
def sortMatchups(matchups):
#     matchups["baseline"] = matchups["wRPI"] < matchups["lRPI"]
    matchups["baseline"] = matchups["w_seed"] < matchups["l_seed"]
    matchups = matchups.apply(executeSeedTiebreaker, axis = 1, result_type = "broadcast")
    matchups["baseline"].replace(False, 0, inplace=True)
    matchups["baseline"].replace(True, 1, inplace=True)
    sortedMatchups = matchups.apply(sortRowBySeed, axis = 1, result_type = "broadcast")

    newColNames = []
    columns = sortedMatchups.columns
    for name in columns:
        if name[0] == 'l':
            newColNames.append("a" + name[1:])
        elif name[0] == "w":
            newColNames.append("b" + name[1:])
        else:
            newColNames.append(name)
    sortedMatchups.columns = newColNames
    return sortedMatchups

def findAllDeltas(matchups):
    # expand results to extract each item from list returned each iteration of apply function
    deltas = matchups.apply(findDeltasForMatch, axis = 1, result_type = "expand")
    colNames = list(matchups.columns[0:6]) + ["DRB", "EFG", "FTA", "FTP", "MOL", "MOV", "ORB", "POSS", "RPI",
                                             "TO", "TOF", "confTourneyWins", "dEff", "numGamesPlayed", "oEff",
                                             "winsVsTourney", "seed", "baseline"]
    deltas.columns = colNames
    return deltas

def mergeRankings(matchups):
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["id"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    temp = matchups.merge(seeds, how='left', left_on = "w_id", right_on = "id")
    temp = temp.rename(index=str, columns={"Seed": "w_seed"})
    temp = temp.merge(seeds, how='left', left_on = "l_id", right_on = "id")
    temp = temp.rename(index=str, columns = {"Seed": "l_seed"})
    merged = temp.drop(["id_x", "id_y"], axis = 1)
    return merged

In [57]:
import pandas as pd 
import numpy as np
import team, game as g
import Model as mod
from sklearn.ensemble import RandomForestClassifier
# Used for developing visual of Random Forest if desired
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

def getTeamNames():
    """
    Return dictionary where key is team ID and value is team name
    """
    names = {}
    teams = pd.read_csv("Data/Teams.csv")
    for index, row in teams.iterrows():
        teamId = row["TeamID"]
        name = row["TeamName"]
        names[teamId] = name
    return names

def getSeasonStats(ncaaTourneyTeams):
    """
    Use regular season results and RPI rankings to create a 
    dictionary where key is the team's ID and the value is a 
    Team object. Team objects contain yearly avg stats for each 
    team in various categories.
    """
    teams = {}
    names = getTeamNames()
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    for index, row in regSeasonResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        wRPI = None
        lRPI = None
        try:
            wRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == wTeamId)].iloc[0]["OrdinalRank"]
            lRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == lTeamId)].iloc[0]["OrdinalRank"]
        except Exception as e:
            pass
            # print str(lTeamId) + " " + str(season) + " not found"
        
        if customWId not in teams:
            teams[customWId] = team.Team(customWId)
        if customLId not in teams:
            teams[customLId] = team.Team(customLId)
        wTeam = teams[customWId]
        wTeam.RPI = wRPI
        wTeam.name = names[wTeamId]
        wTeam.updateStats(row, True)
        if customLId in ncaaTourneyTeams:
            wTeam.winsVsTourney += 1
        lTeam = teams[customLId]
        lTeam.name = names[lTeamId]
        lTeam.RPI = lRPI
        lTeam.updateStats(row, False)
    return teams

def populateNCAATourneyTeams():
    """
    Create an ID for each team using a combination of its id and the season the team played in. 
    Output a dictionary with an entry for each team whose key is its newly created id
    """
    ncaaTourneyTeams = {}
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customWId] = 1
        if customLId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customLId] = 1
    return ncaaTourneyTeams

def getMatchups(teams):
    """
    Use NCAA Tournament results to return data frame of matchups where each row contains data for one matchup between two teams, including their yearly avg totals in statistical categories, RPI, and game result.
    """
    matchups = []
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId in teams and customLId in teams:
            wTeamData = teams[customWId].objToDict().copy()
            for key in wTeamData.keys():
                if key[0] == "w":
                    wTeamData[key] = wTeamData[key]
                else:
                    wTeamData["w" + key] = wTeamData[key]
                del wTeamData[key]
            lTeamData = teams[customLId].objToDict().copy()
            for key in lTeamData.keys():
                if key[0] == "l":
                    lTeamData[key] = lTeamData[key]
                else:
                    lTeamData["l" + key] = lTeamData[key]
                del lTeamData[key]
            matchupData = wTeamData.copy()
            matchupData.update(lTeamData)
            matchupData["dayNum"] = dayNum
            matchupData["season"] = season
            matchups.append(matchupData)
    colOrder = ["dayNum", "season", "l_id", "lname", "w_id", "wname", "lDRB", "lEFG", "lFTA", "lFTP", "lMOL", "lMOV", "lORB", "lPOSS",
                "lRPI", "lTO", "lTOF", "lconfTournWins", "ldEff", "lnumGamesPlayed", "loEff", "lwinsVsTourney",
                "wDRB", "wEFG", "wFTA", "wFTP", "wMOL", "wMOV", "wORB", "wPOSS", "wRPI", "wTO", "wTOF", 
                "wconfTournWins", "wdEff", "wnumGamesPlayed", "woEff", "wwinsVsTourney"]
    df = pd.DataFrame.from_dict(matchups)
    df = df[colOrder]
    return df

def getMatchupData():
    """
    Returns data frame of historical matchups in NCAA tournament.
    Reads in existing CSV if available. Otherwise, produces data frame by creating Team objects, calculating yearly avg totals for each team, and joining with historical NCAA tourney matchup data
    """
    try:
        matchups = pd.read_csv("Data/output/matchups.csv")
        return matchups
    except Exception as e:
        ncaaTourneyTeams = populateNCAATourneyTeams()
        teamObjs = getSeasonStats(ncaaTourneyTeams)
        matchups = getMatchups(teamObjs)
        matchups.to_csv("Data/output/matchups.csv", index=False)
        return matchups

In [46]:
    matchups = getMatchupData()
    seededMatches = mergeRankings(matchups)
    sortedMatches = sortMatchups(seededMatches)

,dayNum,season,a_id,aname,b_id,bname,aDRB,aEFG,aFTA,aFTP,...,bTO,bTOF,bconfTournWins,bdEff,bnumGamesPlayed,boEff,bwinsVsTourney,b_seed,a_seed,baseline
880,137,2016,1421_2016,UNC Asheville,1437_2016,Villanova,25.4516,0.515873,24.3871,0.688113,...,11.1176,13.8529,3,95.4101,34,115.005,10,2,15,1
881,137,2016,1338_2016,Pittsburgh,1458_2016,Wisconsin,24.9677,0.546296,21.2903,0.757027,...,10.9688,12.1562,0,102.536,32,109.396,9,7,10,1
882,137,2016,1451_2016,Weber St,1462_2016,Xavier,28.4516,0.635417,24.5806,0.682579,...,12.75,14.125,2,102.544,32,117.45,10,2,15,1
883,138,2016,1463_2016,Yale,1181_2016,Duke,26.8148,0.688679,23.5556,0.657093,...,9.87879,12.1818,1,105.55,33,119.138,6,4,12,1
884,138,2016,1428_2016,Utah,1211_2016,Gonzaga,28.9062,0.755319,20.6875,0.746049,...,12.1515,10.3636,3,102.406,33,113.462,11,3,11,0
885,138,2016,1246_2016,Kentucky,1231_2016,Indiana,25.3438,0.537736,19.4062,0.724854,...,11,12.3529,4,100.709,34,117.561,3,4,5,0
886,138,2016,1114_2016,Ark Little Rock,1235_2016,Iowa St,24.1935,0.625,17.129,0.727142,...,11.5625,12.4062,0,102.524,32,111.618,9,4,12,1
887,138,2016,1163_2016,Connecticut,1242_2016,Kansas,27.0588,0.635417,17.0882,0.784901,...,12.5455,13.4545,4,95.1319,33,112.923,16,1,9,1
888,138,2016,1455_2016,Wichita St,1274_2016,Miami FL,25.871,0.357143,21.9032,0.710515,...,10.6562,11.625,1,100.121,32,113.082,8,3,11,1
889,138,2016,1344_2016,Providence,1314_2016,North Carolina,24.9091,0.656863,22.5455,0.718468,...,10.9118,13.1471,4,98.2943,34,116.211,10,1,9,1
